**Import necessary libraries**

In [2259]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2260]:
df_name = "EtsySoldOrderItems2020"

**Read csv files**

In [2261]:
df = pd.read_csv(df_name+".csv")

Columns of dataframe

In [2262]:
df.columns

Index(['Sale Date', 'Item Name', 'Buyer', 'Quantity', 'Price', 'Coupon Code',
       'Coupon Details', 'Discount Amount', 'Shipping Discount',
       'Order Shipping', 'Order Sales Tax', 'Item Total', 'Currency',
       'Transaction ID', 'Listing ID', 'Date Paid', 'Date Shipped',
       'Ship Name', 'Ship Address1', 'Ship Address2', 'Ship City',
       'Ship State', 'Ship Zipcode', 'Ship Country', 'Order ID', 'Variations',
       'Order Type', 'Listings Type', 'Payment Type', 'InPerson Discount',
       'InPerson Location', 'VAT Paid by Buyer', 'SKU'],
      dtype='object')

In [2263]:
df = df.rename(columns={"Item Name":"Item Model"})

In [2264]:
df.head(2)

,Sale Date,Item Model,Buyer,Quantity,Price,Coupon Code,Coupon Details,Discount Amount,Shipping Discount,Order Shipping,...,Ship Country,Order ID,Variations,Order Type,Listings Type,Payment Type,InPerson Discount,InPerson Location,VAT Paid by Buyer,SKU
0,12/31/20,Genuine Gray Leather Samsung Galaxy Note 20 Pr...,Shanon Browne (wj1ol9t2d47zhqco),1,44.67,25OFFDECEMBER,25OFFDECEMBER - % off,11.17,0.00,0.0,...,United States,1908770705,Personalization Request:Add Laser Engraving,online,listing,online_cc,NaN,NaN,0,BO-01-FXCP-TN18E-0NT20-00
1,12/31/20,Genuine Leather Apple iPhone 12 & iPhone 12 Pr...,Kevin Butterfield (kevinbutterfield72),1,52.00,25OFFDECEMBER,25OFFDECEMBER - % off,13.00,3.21,0.0,...,United States,1900438126,Personalization Request:No Personalization,online,listing,online_cc,NaN,NaN,0,BO-01-FXC0-RS02E-IP121-00


Check null values

In [2265]:
df.isna().sum()

Sale Date              0
Item Model             0
Buyer                124
Quantity               0
Price                  0
Coupon Code           21
Coupon Details        21
Discount Amount        0
Shipping Discount      0
Order Shipping         0
Order Sales Tax        0
Item Total             0
Currency               0
Transaction ID         0
Listing ID             0
Date Paid              0
Date Shipped           0
Ship Name              0
Ship Address1          0
Ship Address2        562
Ship City              0
Ship State            73
Ship Zipcode           4
Ship Country           0
Order ID               0
Variations            27
Order Type             0
Listings Type          0
Payment Type           0
InPerson Discount    702
InPerson Location    702
VAT Paid by Buyer      0
SKU                    0
dtype: int64

In [2266]:
df.drop(columns=['Buyer','Coupon Details','Shipping Discount','Order Shipping',\
                      'Order Sales Tax','Item Total', 'Currency','Transaction ID', 'Listing ID',\
                      'Date Paid', 'Date Shipped','Ship Name', 'Ship Address1', 'Ship Address2',\
                      'Ship Zipcode', 'Variations','Order Type', 'Listings Type',\
                      'Payment Type', 'InPerson Discount','InPerson Location', 'VAT Paid by Buyer',\
                      'SKU','Ship City','Ship State'],inplace = True)

In [2267]:
df.dropna(inplace=True)

Check duplicates

In [2268]:
df.duplicated().all().sum()

0

Map seasons

In [2269]:
def map_seasons(dataframe,season,sale_date):
    dataframe[season] = ""  # Add a new column named "Season" to the dataframe
    for i in dataframe[sale_date]:
        nums = str.split(i,sep="/")
        date = ""
        if nums[0] in ["12","01","02"]:
            date = "Winter"
        elif nums[0] in ["03","04","05"]:
            date = "Spring"
        elif nums[0] in ["06","07","08"]:
            date = "Summer"
        elif nums[0] in ["09","10","11"]:
            date = "Fall"
        dataframe.loc[dataframe[sale_date] == i, season] = date 
    return dataframe   

In [2270]:
df = map_seasons(df,"Season","Sale Date")

In [2271]:
seasons_dict = {'Winter': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3}
df['Season'] = df['Season'].replace(seasons_dict)

In [2272]:
df['Sale Date'] = pd.to_datetime(df['Sale Date'], format='%m/%d/%y')
df['Day Of Year'] = df['Sale Date'].apply(lambda x: x.timetuple().tm_yday)

In [2273]:
countries = df['Ship Country'].unique()
countries

array(['United States', 'Switzerland', 'Germany', 'Canada',
       'United Kingdom', 'Australia', 'The Netherlands', 'Luxembourg',
       'Sweden', 'France', 'Latvia', 'Israel', 'Greece', 'Italy',
       'Ireland', 'New Zealand', 'Finland', 'South Africa', 'Reunion',
       'Denmark', 'Austria', 'United Arab Emirates', 'Hong Kong',
       'Norway', 'Malaysia', 'Taiwan', 'Estonia', 'Philippines'],
      dtype=object)

In [2274]:
country_map = {'United States':1, 'Canada':2, 'United Kingdom':3, 'Australia':4,
       'Switzerland':5, 'Sweden':6, 'Israel':7, 'Italy':8, 'Ireland':9,
       'New Zealand':10, 'United Arab Emirates':11, 'Hong Kong':12, 'Malaysia':13,
       'The Netherlands':14, 'Taiwan':15, 'Germany':16, 'Philippines':17,"Austria":18,
              "South Korea":19,"France":20,"Hungary":21,"Singapore":22,"Finland":23,
              "Belgium":24,"Luxembourg":25,"Puerto Rico":26,"Sri Lanka":27,"South Africa":28,
              "Latvia":29,"Greece":30,"Reunion":31,"Denmark":32,"Norway":33,"Estonia":34,
              "Spain":35,"Brazil":36,"India":37,"Japan":38,"Indonesia":39,"Mexico":40,
              "Russia":41}

In [2275]:
def map_country(country):
    x = False
    for i, number in country_map.items():
        if i in country:
            x= True
            return number
    if x == False:
        print(country)
df["Ship Country"] = df["Ship Country"].apply(map_country)

In [2276]:
# df['Ship Country'] = df['Ship Country'].map(country_map)

In [2277]:
coupons = df['Coupon Code'].unique()
coupons

array(['25OFFDECEMBER', 'CHRISTMAS40OFF', 'CHRISTMAS40OFF;CHRISTMAS40OFF',
       '25OFFNOVEMBER', 'ABONDONED25', '2ORMOREITEMS30OFF',
       '20OFFOCTOBER', '2ORMOREITEMS25OFF', '20OFFSEPTEMBER', 'FAV25',
       'SEPTEMBER20OFF', '20OFFAUGUST', 'TWOITEMS30OFF', 'JULY20OFF',
       'JUNE20OFF', 'MAY15OFF', 'APRIL20OFF', '20OFF', 'GETTWO20',
       'FEB10', 'JAN10', 'JANUARY15', 'CHRISTMAS25', 'ABANDONED30'],
      dtype=object)

In [2278]:
coupons_map ={'25OFFDECEMBER':1, 'CHRISTMAS40OFF':2, 'CHRISTMAS40OFF;CHRISTMAS40OFF':2,
       '25OFFNOVEMBER':3, 'ABONDONED25':4, '2ORMOREITEMS30OFF':5,
       '20OFFOCTOBER':6, '2ORMOREITEMS25OFF':7, '20OFFSEPTEMBER':8, 'FAV25':9,
       'SEPTEMBER20OFF':8, '20OFFAUGUST':9, 'TWOITEMS30OFF':10, 'JULY20OFF':11,
       'JUNE20OFF':12, 'MAY15OFF':13, 'APRIL20OFF':14, '20OFF':15, 'GETTWO20':16,
       'FEB10':17, 'JAN10':18, 'JANUARY15':19, 'CHRISTMAS25':20, 'ABANDONED30':21,
        '25OFFF':22,'2ORMOREITEMS40OFF':23,'25OFF':22,'2OREMOREITEMS40OFF;2OREMOREITEMS40OFF':23,
        '2OREMOREITEMS40OFF;2OREMOREITEMS40OFF;2OREMOREITEMS40OFF':24,'25OFFJANUARY':25,
            '2ORMOREITEMS40PFF':26,'BIGSALEOFF':27, 'FAVORITED20':28,'THANKYOU':29,'SEPTEMBER25OFF':30,
             '2ORMOREITEMS30OFF':31,'25OFFSUMMER':32,'JUNE25OFF':33,'MAY25OFF':34,'MAY25OFF;MAY25OFF':34,
             'APRIL25OFF':35,'MOTHERSDAY':36,'NOTES20':37,'25PROMO':38,'ALL50':39,'20OFFJANUARY':40}

In [2279]:
df['Coupon Code'] = df['Coupon Code'].map(coupons_map)

In [2280]:
df.drop(columns=['Sale Date','Day Of Year'],inplace = True)

In [2281]:
color_map = {"Brown":1,"Gray":2,"Black":3,"Navy Blue":4,"Blue":5,"Yellow":6,"Red":7,"Pink":8,
            "Multicolered":9,"Green":10,"Beige":11,"Purple":12,"Leopar":13,"Multicolored":9,
            "Grey":2,"Multicolor":9,"Rose Gold":14,"Rainbow":15,"White":16}

In [2282]:
df.columns

Index(['Item Model', 'Quantity', 'Price', 'Coupon Code', 'Discount Amount',
       'Ship Country', 'Order ID', 'Season'],
      dtype='object')

In [2283]:
def map_color(item_name):
    x = False
    for model, number in color_map.items():
        if model in item_name:
            x= True
            return number
    if x == False:
        if "Genuine" in item_name:
            return 0
        print(item_name)
df["Color"] = df["Item Model"].apply(map_color)

Leather Apple iPhone 12 & iPhone 12 Pro (6.1") Magnetic Detachable RFID Protection Wallet Case with Card Slots, iPhone 12 Pro Back Cover
Leather Apple iPhone 12 PRO MAX (6.7") Magnetic Detachable RFID Protection Pouch Wallet Case with Card Slots, iPhone 12 Pro Max Back Cover
Leather Apple iPhone 12 & iPhone 12 Pro (6.1") Magnetic Detachable RFID Protection Wallet Case with Card Slots, iPhone 12 Pro Back Cover
Leather Apple iPhone 12 PRO MAX (6.7") Magnetic Detachable RFID Protection Pouch Wallet Case with Card Slots, iPhone 12 Pro Max Back Cover
Nude Leather Apple iPhone 12 PRO MAX (6.7") Magnetic Detachable RFID Protection Wallet Case with Card Slots, iPhone 12 Pro Max Back Cover
Leather Apple iPhone 12 & iPhone 12 Pro (6.1") Magnetic Detachable RFID Protection Wallet Case with Card Slots, iPhone 12 Pro Back Cover
Leather Apple iPhone 12 PRO MAX (6.7") Magnetic Detachable RFID Protection Pouch Wallet Case with Card Slots, iPhone 12 Pro Max Back Cover
Leather Apple iPhone 12 & iPhone 1

In [2284]:
df['Color'].fillna(0, inplace=True)

In [2285]:
model_map = {"iPhone 12": 1, "iPhone 12 Pro": 2, "Samsung Galaxy S21": 3,"Galaxy Note 20":4,\
             "iPhone 11 Pro Max": 5,"iPhone 12 PRO MAX":6,"Watch Band":7,"iPhone Mini 12":8,\
            "Galaxy Note 20 ULTRA":9,"Galaxy S20 Ultra":10,"iPhone 13 Series":11,
            "Galaxy Note 10 Plus":12,"iPhone 13 Pro":13,"Dog":14,"AirPods":14,"Desk Mat":15,
            "Galaxy S20 Plus":16,"iPhone SE":17,"Samsung Galaxy S20":18,"Apple iPhone XS MAX":19,
            "Apple iPhone XR":20,"Apple iPhone X/XS":21,"iPhone 11":22,"Samsung S10":23,
            "Card Holder":24,"Galaxy N10":25,"iPhone 7/8":26,"Women's Wallet":27,
            "Custom order":28,"Organizer":29,"iPhone 14 Series":30,"Galaxy S22 Series":31,"AirTag":32,
            "Mouse Pad":33,"Galaxy S22":34,"Makeup Purse":35,"Huawei Mate 20 Pro":36,
             "Samsung Galaxy S10":37,"Huawei Mate 20 Lite":38,"Samsung Galaxy S8 Plus":39,
             "Samsung Galaxy S10 Plus":40,"Samsung Note 8":41,"Huawei P20 Pro":42,"Samsung Note 10":43,
            "Samsung Galaxy S8":44,"Galaxy S9 Plus":45,"Galaxy Note 9":46,"Galaxy S9":47,
            "iPhone XS MAX":48,"Samsung Note 9":46,"iPhone X/iPhone XS":49,"iPhone X / iPhone XS":49,
            "Apple Watch":7,"iPhone X/XS":49,"iPhone 7 / 8":26,"Galaxy Note 10+":51,"iPhone X":52} 

In [2286]:
def map_model(item_name):
    x = False
    for model, number in model_map.items():
        if model in item_name:
            x= True
            return number
    if x == False:
        print(item_name)
df["Item Model"] = df["Item Model"].apply(map_model)

In [2287]:
df.sample(2)

,Item Model,Quantity,Price,Coupon Code,Discount Amount,Ship Country,Order ID,Season,Color
427,43,1,36.6,11,7.32,1,1692787868,2,1.0
155,1,1,132.0,3,33.00,1,1835414023,3,0.0


In [2288]:
df.describe()

,Item Model,Quantity,Price,Coupon Code,Discount Amount,Ship Country,Order ID,Season,Color
count,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,6.810000e+02,681.000000,681.000000
mean,16.753304,1.005874,60.315301,8.898678,13.180749,3.099853,1.739061e+09,1.729809,2.970631
std,12.762569,0.076471,18.859632,5.201528,7.727264,5.365085,9.822614e+07,1.118221,3.280381
min,1.000000,1.000000,25.330000,1.000000,0.000000,1.000000,1.557283e+09,0.000000,0.000000
25%,5.000000,1.000000,49.000000,3.000000,8.760000,1.000000,1.655229e+09,1.000000,1.000000
50%,17.000000,1.000000,56.400000,9.000000,11.330000,1.000000,1.739930e+09,2.000000,1.000000
75%,23.000000,1.000000,65.400000,13.000000,14.670000,2.000000,1.824392e+09,3.000000,5.000000
max,52.000000,2.000000,146.250000,31.000000,83.460000,34.000000,1.908771e+09,3.000000,16.000000


In [2289]:
df.to_csv(df_name+"Modified.csv", index=False)

In [2290]:
df_2020 = pd.read_csv("EtsySoldOrderItems2020Modified.csv")
df_2021 = pd.read_csv("EtsySoldOrderItems2021Modified.csv")
df_2022 = pd.read_csv("EtsySoldOrderItems2022Modified.csv")

In [2291]:
merged_df = pd.concat([df_2020, df_2021, df_2022])

In [2292]:
merged_df.describe()

,Item Model,Quantity,Price,Coupon Code,Discount Amount,Ship Country,Order ID,Season,Color
count,2227.000000,2227.000000,2227.000000,2173.000000,2227.000000,2227.000000,2.227000e+03,2227.000000,2227.000000
mean,12.615177,1.003592,71.017975,19.782789,17.821230,2.750786,2.106016e+09,1.550516,2.461159
std,10.496164,0.059841,29.852191,10.763177,10.719705,5.052318,3.085715e+08,1.123359,3.585773
min,1.000000,1.000000,25.330000,1.000000,0.000000,1.000000,1.557283e+09,0.000000,0.000000
25%,4.000000,1.000000,52.000000,11.000000,12.010000,1.000000,1.841292e+09,1.000000,0.000000
50%,11.000000,1.000000,64.000000,22.000000,15.630000,1.000000,2.120118e+09,2.000000,1.000000
75%,18.000000,1.000000,76.160000,25.000000,19.645000,1.000000,2.345903e+09,3.000000,3.000000
max,52.000000,2.000000,292.000000,40.000000,126.000000,41.000000,2.743856e+09,3.000000,16.000000


In [2293]:
merged_df_csv = merged_df.drop(columns=["Order ID"])

In [2294]:
merged_df_csv.to_csv("EtsySoldOrderItems2020-2021-2022.csv", index=False)

In [2295]:
merged_df.isna().sum()

Item Model          0
Quantity            0
Price               0
Coupon Code        54
Discount Amount     0
Ship Country        0
Order ID            0
Season              0
Color               0
dtype: int64

In [2296]:
merged_df.duplicated().all().sum()

0

In [2297]:
review_df = pd.read_json('reviews.json')

In [2298]:
review_df.head()

,reviewer,date_reviewed,star_rating,message,order_id
0,Katie,01/24/2023,5,"Bought for my bf, arrived earlier than expecte...",2698975905
1,Andrea,01/02/2023,4,Wir haben den Artikel in schwarz und blau best...,2735309118
2,Andrea,01/02/2023,4,Die Handytaschen sind sehr hochwertig verarbei...,2735309118
3,Lerissa,12/30/2022,4,,2732670357
4,Lerissa,12/30/2022,4,,2732670357


In [2299]:
review_df['message'] = np.where(review_df['message'] == '', 0, 1)

In [2300]:
etsy = pd.merge(merged_df, review_df, left_on='Order ID',right_on='order_id')

In [2301]:
etsy.drop(columns=['date_reviewed',"order_id","Order ID","reviewer"],inplace = True)

In [2302]:
etsy.head()

,Item Model,Quantity,Price,Coupon Code,Discount Amount,Ship Country,Season,Color,star_rating,message
0,5,1,55.33,1.0,13.83,1,0,1.0,2,1
1,1,1,58.67,1.0,14.67,5,0,0.0,5,0
2,1,1,62.00,1.0,15.50,1,0,3.0,5,1
3,1,1,52.00,1.0,13.00,1,0,1.0,5,1
4,4,1,115.00,2.0,46.00,1,0,0.0,5,1


In [2303]:
etsy.describe()

,Item Model,Quantity,Price,Coupon Code,Discount Amount,Ship Country,Season,Color,star_rating,message
count,538.000000,538.000000,538.000000,509.000000,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000
mean,11.951673,1.005576,70.794870,20.829077,18.315911,2.827138,1.570632,2.698885,4.657993,0.723048
std,10.259459,0.074535,26.200987,10.258675,12.007004,5.167389,1.069134,3.754877,0.900104,0.447909
min,1.000000,1.000000,25.330000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,3.000000,1.000000,53.025000,13.000000,12.252500,1.000000,1.000000,0.000000,5.000000,0.000000
50%,10.000000,1.000000,64.750000,22.000000,16.000000,1.000000,2.000000,1.000000,5.000000,1.000000
75%,18.000000,1.000000,77.160000,30.000000,20.000000,1.000000,2.000000,5.000000,5.000000,1.000000
max,51.000000,2.000000,206.670000,40.000000,95.880000,41.000000,3.000000,16.000000,5.000000,1.000000


In [2304]:
etsy.to_csv("EtsySoldOrderItemsAndReviews2020-2021-2022.csv", index=False)